In [1]:
# CORRIDA # 1. MODELO MLP CON UNA CAPA OCULTA

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import os
from math import sqrt

# Cargar el script de neuronas
neuronas_script_path = 'C:/Mae_BI_UAM/M13/RN_Final/Evasion/libs/neuronas.py'
exec(open(neuronas_script_path).read())

# Cargar los datos
data_path = 'C:/Mae_BI_UAM/M13/RN_Final/Evasion/data/data.csv'
data = pd.read_csv(data_path)

# Mostrar las primeras filas del conjunto de datos
print(data.head())

# Convertir la variable categórica 'GRUPO' a factores
label_encoders = {}
for column in data.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# Separar las características y la variable objetivo
X = data.drop(columns=['AJUSTE'])
y = data['AJUSTE']

# Normalizar los datos
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Separar el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Función para calcular las métricas de rendimiento
def calcular_metricas(y_real, y_pred):
    mse = mean_squared_error(y_real, y_pred)
    rmse = sqrt(mse)
    mae = mean_absolute_error(y_real, y_pred)
    return mse, rmse, mae

# Escenario base: 20 neuronas, 0.05 tasa de aprendizaje, 500 épocas
escenario_base = (20, 0.05, 500)
RNA_MLP_base = MLP([X_train.shape[1], escenario_base[0], 1])
RNA_MLP_base.train(X_train, y_train.values, escenario_base[1], escenario_base[2])

predicciones_train_base = RNA_MLP_base.feedforward(X_train).flatten()
predicciones_test_base = RNA_MLP_base.feedforward(X_test).flatten()

mse_train_base, rmse_train_base, mae_train_base = calcular_metricas(y_train, predicciones_train_base)
mse_test_base, rmse_test_base, mae_test_base = calcular_metricas(y_test, predicciones_test_base)

print(f"Escenario Base: Neuronas={escenario_base[0]}, Tasa de aprendizaje={escenario_base[1]}, Épocas={escenario_base[2]}")
print(f"RMSE Train: {rmse_train_base}, RMSE Test: {rmse_test_base}")
print(f"MAE Train: {mae_train_base}, MAE Test: {mae_test_base}\n")

# Resto de los escenarios con tasas de aprendizaje adicionales
tasas_aprendizaje = [0.05, 0.025, 0.01]
configuraciones = [(8, tasa, 50) for tasa in tasas_aprendizaje] + \
                  [(12, tasa, 50) for tasa in tasas_aprendizaje] + \
                  [(18, tasa, 50) for tasa in tasas_aprendizaje] + \
                  [(25, tasa, 50) for tasa in tasas_aprendizaje] + \
                  [(30, tasa, 50) for tasa in tasas_aprendizaje] + \
                  [(8, tasa, 50) for tasa in tasas_aprendizaje] + \
                  [(12, tasa, 100) for tasa in tasas_aprendizaje] + \
                  [(18, tasa, 100) for tasa in tasas_aprendizaje] + \
                  [(20, tasa, 100) for tasa in tasas_aprendizaje] + \
                  [(25, tasa, 100) for tasa in tasas_aprendizaje] + \
                  [(30, tasa, 100) for tasa in tasas_aprendizaje]

resultados = [(0, escenario_base[0], escenario_base[1], escenario_base[2], mse_train_base, rmse_train_base, mae_train_base, mse_test_base, rmse_test_base, mae_test_base)]
predicciones_totales = [pd.DataFrame({
    'Escenario': [0] * len(y_test),
    'Valor Real': y_test.values,
    'Predicción': predicciones_test_base
})]

for i, (neuronas, tasa_aprendizaje, epocas) in enumerate(configuraciones, start=1):
    RNA_MLP = MLP([X_train.shape[1], neuronas, 1])
    RNA_MLP.train(X_train, y_train.values, tasa_aprendizaje, epocas)
    predicciones_train = RNA_MLP.feedforward(X_train).flatten()
    predicciones_test = RNA_MLP.feedforward(X_test).flatten()

    mse_train, rmse_train, mae_train = calcular_metricas(y_train, predicciones_train)
    mse_test, rmse_test, mae_test = calcular_metricas(y_test, predicciones_test)

    resultados.append((i, neuronas, tasa_aprendizaje, epocas, mse_train, rmse_train, mae_train, mse_test, rmse_test, mae_test))
    predicciones_totales.append(pd.DataFrame({
        'Escenario': [i] * len(y_test),
        'Valor Real': y_test.values,
        'Predicción': predicciones_test
    }))

    print(f"Escenario {i}: Neuronas={neuronas}, Tasa de aprendizaje={tasa_aprendizaje}, Épocas={epocas}")
    print(f"RMSE Train: {rmse_train}, RMSE Test: {rmse_test}")
    print(f"MAE Train: {mae_train}, MAE Test: {mae_test}\n")

         AJUSTE  TIEMPO           CIF          KILO           PAG  GRUPO
0  5.532099e+06    8.35  7.376132e+07  1.622749e+07  1.844033e+07     -1
1  3.016764e+06   67.00  4.022351e+07  8.849173e+06  1.005588e+07     -1
2  2.843415e+06   23.46  3.791220e+07  8.340684e+06  9.478050e+06     -1
3  2.529744e+06   20.25  3.372992e+07  7.420583e+06  8.432481e+06      1
4  1.899444e+06    3.40  2.532592e+07  5.571702e+06  6.331480e+06     -1
Escenario Base: Neuronas=20, Tasa de aprendizaje=0.05, Épocas=500
RMSE Train: 19570.559537394587, RMSE Test: 12960.125495852153
MAE Train: 1277.953165199977, MAE Test: 1267.190424828549

Escenario 1: Neuronas=8, Tasa de aprendizaje=0.05, Épocas=50
RMSE Train: 19570.559537394587, RMSE Test: 12960.125495852153
MAE Train: 1277.953165199977, MAE Test: 1267.190424828549

Escenario 2: Neuronas=8, Tasa de aprendizaje=0.025, Épocas=50
RMSE Train: 19570.559537394587, RMSE Test: 12960.125495852153
MAE Train: 1277.953165199977, MAE Test: 1267.190424828549

Escenario 